# Identificação

**Nome:** Guilherme Albarrans Leite

**NUSP:** 11890922

# MAC0434 e MAC5788 - Planejamento e Aprendizado por Reforço (2023)


## Aprendizado por reforço profundo

Nesse notebook iremos implementar o algoritmo de controle DQN, um algoritmo de aprendizado por reforço profundo, que combina redes neurais e programação dinâmica aproximada. A proposta geral do DQN é combinar a regra de atualização do Q-Learning com aproximadores de função não-lineares para se obter boa generalização sobre o conjunto de possíveis observações durante episódios de interação do agente com o ambiente.

<img src="https://github.com/thiagopbueno/curso-verao-rl-ime-2021/blob/master/notebooks/aula3/img/control.png?raw=true" alt="Agent-Env Loop" style="width: 300px;"/>

Neste notebook o nosso objetivo será resolver o ambiente `CartPole-v1`.

### Objetivos:

- Entender o papel da otimalidade de Bellman para algoritmos de controle
- Desenvolver intuição sobre o problema de exploração/explotação em RL
- Ter um primeiro contato com técnicas de treinamento de algoritmos de deep RL
- Familiarizar-se com a biblioteca de redes neurais dm-sonnet

### 💻 Configurações do Ambiente

O código utilizado na aula prática foi desenvolvido em 2021. É importante observar que as versões das bibliotecas utilizadas não são necessariamente as versões mais recentes, e o uso de bibliotecas com versões distintas das especificadas pode resultar em conflitos no código.


A versão do Python utilizada é a 3.7, e as versões das bibliotecas estão especificadas no arquivo "requirements.txt".

Para garantir que não haja conflitos com o código disponibilizado, sugerimos a instalação dos pacotes e bibliotecas utilizados em algum ambiente virtual de sua escolha. Uma possibilidade é criar um ambiente virtual utilizando o ```virtualenv``` conforme instruções disponíveis no Moodle.

# 💻 Execução do Código

Após configurar o ambiente, execute a célula abaixo que importa as bibliotecas necessárias.

In [14]:
from collections import deque
from datetime import datetime
import os.path as osp
import time

import gym
from gym.wrappers import AtariPreprocessing, FrameStack, Monitor, TimeLimit
import numpy as np
import sonnet as snt
import tensorflow as tf
from tqdm.notebook import trange

from utils import logging
from utils.nn import initializers
from utils import replay
from utils import schedule
from utils import tf_utils

#tf_utils.set_tf_allow_growth() # necessário apenas se você dispõe de GPU

## Ambiente - CartPole-v1

Como veremos neste exercício-programa, é sempre uma boa ideia em aprendizado por reforço iniciar o estudo de um algoritmo por um problema simples e pequeno para o qual você poderá resolver em poucos minutos. Para isso, o ambiente do `CartPole-v1` é usualmente um dos primeiros problemas que um agente baseado em aprendizado por reforço deve ser capaz de resolver antes de tentar atacar problemas mais complexos.

> Obs. Note que, por se tratar de uma implementação de 2021, a biblioteca utilizada para a importação do ambiente do CartPole é a `gym` ao invés da biblioteca [gymnasium](https://gymnasium.farama.org/index.html), que foi utilizada no EP2 (2023).

In [15]:
def make_envs(env_id, n=20): 
    env = gym.make(env_id) # ambiente que será usado no treinamento
    eval_env = gym.vector.make(env_id, num_envs=n, asynchronous=True) # ambiente de avaliação
    test_env = gym.make(env_id) # ambiente de teste
    return env, eval_env, test_env

In [16]:
env, eval_env, test_env = make_envs("CartPole-v1", n=10)# originalmente era n=20, mas não terminava a execussão mesmo após 7h rodando

## Deep Q-Learning

Como visto em aula o algoritmo `DQN` procura aproximar a função $Q(s, a)$ utilizando redes neurais treinadas por meio da otimização de uma função objetivo baseada na regra de atualização do Q-Learning. O algoritmo abaixo descreve de maneira geral o treinamento de um agente `DQN`.

<img src="https://github.com/thiagopbueno/curso-verao-rl-ime-2021/blob/master/notebooks/aula3/img/dqn-algo.png?raw=true"/>

Nessa seção desenvolveremos os componentes deste algoritmo passo a passo:

1. **Redes neurais (networks)**: inicialmente construiremos a rede neural para a função $Q(s, a)$, usando a biblioteca `dm-sonnet`;
2. **Função objetivo (loss)**: uma vez definida a classe da função $Q(s, a)$, implementaremos a função objetivo utilizada no problema de "regressão" que o Q-Learning tenta resolver;
3. **Atualização (update)**: em seguida instanciaremos um otimizador baseado em gradientes que será responsável por minimizar a função objetivo previamente definida; e
4. **Política $\epsilon$-greedy**: por fim, definiremos a política estocástica para exploração.

### Redes Neurais (networks)

Para representar funções $Q(s,a)$ utilizando redes neurais, temos em geral 2 opções de implementação:
1. Definir uma rede com entrada $(s, a)$ e saída um único número real: $Q_\phi : \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$; ou
2. Definir como entrada apenas o estado $s$ e saída um vetor de tamanho $|\mathcal{A}|$: $Q_\phi : \mathcal{S} \rightarrow \mathbb{R}^{|\mathcal{A}|}$ .

Em geral, para algoritmos baseados no `DQN` é costume utilizar a 2a opção.

> **Observação**: note que $\phi \in \mathbb{R}^d$ com $d \ll |S|$, onde $\phi$ denota o conjunto de parâmetros (i.e., *kernels* e *biases*) da rede neural. Dessa forma, a rede deve extrair apenas informações essenciais sobre o estado para a predição do retorno esperado (como vimos na aula de predição).

<img src="https://github.com/thiagopbueno/curso-verao-rl-ime-2021/blob/master/notebooks/aula3/img/conv-net.png?raw=true"/>

In [17]:
class QNetwork(snt.Module):

    def __init__(self, observation_space, action_space, name="QNetwork"):
        super().__init__(name=name)

        self.observation_space = observation_space
        self.action_space = action_space

        # features
        """Definição das camadas ocultas da rede. 2 camadas com 8 neurônios cada."""
        self._torso = snt.nets.MLP(
            [8, 8],
            activation=tf.nn.relu,
            activate_final=True,
            w_init=initializers.he_initializer(),
            name="MLP"
        )

        # predictor
        """Rede completamente conectada."""
        self._q_values = snt.Linear(action_space.n, name="QValues")

    @tf.function
    def __call__(self, obs):
        """Calcula os Q-values de todas as ações para uma dada `obs`."""
        h = self._torso(obs)
        return self._q_values(h)

    @tf.function
    def action_values(self, obs, actions):
        """Calcula os Q-values de uma única `action` específica para uma dada `obs`."""
        batch_size = tf.shape(obs)[0]
        indices = tf.stack([tf.range(batch_size, dtype=actions.dtype), actions], axis=1)
        q_values = tf.gather_nd(self(obs), indices)
        return q_values

    @tf.function
    def hard_update(self, other):
        """Copia os parâmetros da rede `other` para a rede do objeto."""
        for self_var, other_var in zip(self.trainable_variables, other.trainable_variables):
            self_var.assign(other_var)

### Função objetivo (loss)

Lembre-se que o `DQN` se utiliza da regra de atualização baseado em programação dinâmica aproximada do Q-Learning:

$$
\mathcal{L}(\phi) = \mathbb{E}_{(s, a, r, s') \sim \mathcal{D}} [(Q_{\phi}(s, a) - (r + \gamma \max_{a'} Q_{\bar{\phi}}(s', a')))^2]
$$

> **Observação**: lembre que para compor o "alvo" (target) da regressão usamos a rede target $Q_{\bar{\phi}}$. Conforme vimos na aula teórica, o uso de *target networks* é fundamental para melhorar a estabilidade do treinamento. Caso contrário, toda atualização na direção de $\nabla_{\phi} \mathcal{L}(\phi)$ acabaria por alterar também o valor do "alvo" da regressão, tornando o problema de otimização muito mais complicado!

In [18]:
def make_q_learning_loss(q_net, target_q_net, gamma=0.99):
    """Recebe a rede online `q_net` e a rede `target_q_net` e devolve o loss function do Q-Learning."""

    @tf.function
    def _loss(batch):
        """Recebe um batch de experiências e devolve o valor da função objetivo para esse batch."""
        obs = batch["obs"]
        actions = batch["action"]
        rewards = batch["reward"]
        next_obs = batch["next_obs"]
        terminals = tf.cast(batch["terminal"], tf.float32)

        # predictions
        q_values = q_net.action_values(obs, actions)

        # targets
        next_q_values = tf.reduce_max(target_q_net(next_obs), axis=-1)
        q_targets = tf.stop_gradient(rewards + (1 - terminals) * gamma * next_q_values)

        # loss = tf.reduce_mean((q_values - q_targets) ** 2)
        loss = tf.losses.huber(q_values, q_targets)
        return loss

    return _loss

### Atualização (updates)

Uma vez com *loss function* definida, basta instanciar um otimizador escolhendo uma taxa de aprendizado (i.e., `learning_rate`) rodando a célula abaixo:

In [19]:
def make_update_fn(loss_fn, trainable_variables, learning_rate=1e-3):
    optimizer = snt.optimizers.Adam(learning_rate)

    @tf.function
    def _update_fn(batch):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(trainable_variables)
            loss = loss_fn(batch)

        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply(grads, trainable_variables)

        grads_and_vars = {var.name: (grad, var) for grad, var in zip(grads, trainable_variables)}

        return loss, grads_and_vars

    return _update_fn

### Política $\epsilon$-*greedy*

O último componente do algoritmo `DQN` é sua política utilizada para explorar. No notebook de hoje, implementaremos a política exploratória mais simples.

Como vimos na aula teórica, a política $\epsilon$-*greedy* escolhe uma ação aleatória com probabilidade $\epsilon$ e escolhe a ação gulosa com probabilidade $1 - \epsilon$.

In [20]:
class EpsilonGreedyPolicy:

    def __init__(self, q_net, start_val=1.0, end_val=0.01, start_step=1_000, end_step=10_000):
        self.q_net = q_net

        self._schedule = schedule.PiecewiseLinearSchedule((start_step, start_val), (end_step, end_val))

        self._step = tf.Variable(0., dtype=tf.float32, name="step")
        self._epsilon = tf.Variable(start_val, dtype=tf.float32, name="epsilon")

    def __call__(self, obs):
        """Retorna ação aleatória com probabilidade epsilon, c.c., retorna ação gulosa."""
        self._epsilon.assign(self._schedule(self._step))
        self._step.assign_add(1)

        batch_size = tf.shape(obs)[0]
        action_dim = self.q_net.action_space.n

        random_actions = tf.random.uniform(shape=(batch_size,), minval=0, maxval=action_dim, dtype=tf.int32)
        greedy_actions = tf.argmax(self.q_net(obs), axis=-1, output_type=tf.int32)

        return tf.where(
            self._epsilon > tf.random.uniform(shape=(batch_size,)),
            random_actions,
            greedy_actions
        )

### Agente DQN

Com todos os componentes definidos, estamos finalmente preparados para instanciar um agente `DQN` para o ambiente `CartPole-v1`. Execute a célula abaixo para criar a classe `DQN`.

In [21]:
class DQN:

    def __init__(
        self,
        observation_space,
        action_space,
        gamma=0.99,
        target_update_freq=1000,
        learning_rate=1e-3,
        policy_start_val=1.0,
        policy_end_val=0.01,
        policy_start_step=1_000,
        policy_end_step=10_000,
        checkpoint_dir="ckpt"
    ):
        self.observation_space = observation_space
        self.action_space = action_space

        self.gamma = gamma
        self.target_update_freq = target_update_freq
        self.learning_rate = learning_rate

        self.q_net = QNetwork(self.observation_space, self.action_space, name="QNet")
        self.target_q_net = QNetwork(self.observation_space, self.action_space, name="TargetQNet")

        self.policy = EpsilonGreedyPolicy(self.q_net, start_val=policy_start_val, end_val=policy_end_val, start_step=policy_start_step, end_step=policy_end_step)

        self._ckpt_dir = checkpoint_dir
        self._ckpt = tf.train.Checkpoint(q_net=self.q_net)
        self._ckpt_manager = tf.train.CheckpointManager(self._ckpt, directory=self._ckpt_dir, max_to_keep=1)

        self._step = tf.Variable(0, dtype=tf.int32, name="step")

    def build(self):
        """Cria as variáveis das redes online e target e sincroniza inicialmente."""
        input_spec = tf.TensorSpec(self.observation_space.shape, dtype=tf.float32)
        tf_utils.create_variables(self.q_net, input_spec)
        tf_utils.create_variables(self.target_q_net, input_spec)
        self.target_q_net.hard_update(self.q_net)

    def compile(self):
        """Compila a DQN loss junto com a Q-network."""
        self.update_learner = make_update_fn(
            make_q_learning_loss(self.q_net, self.target_q_net, gamma=self.gamma),
            self.q_net.trainable_variables,
            learning_rate=self.learning_rate
        )

    def step(self, obs, training=True):
        """Escolhe a ação para a observação dada."""
        obs = tf.convert_to_tensor(obs, dtype=tf.float32)
        action = self.policy(obs) if training else tf.argmax(self.q_net(obs), axis=-1)
        return action.numpy()

    def learn(self, batch):
        """Recebe um batch de experiências, atualiza os parâmetros das redes, e devolve algumas métricas."""
        # atualiza q_net
        loss, grads_and_vars = self.update_learner(batch)

        # sincroniza target_q_net
        self._step.assign_add(1)
        if self._step % self.target_update_freq == 0:
            self.target_q_net.hard_update(self.q_net)

        # métricas de monitoramento
        stats = {
            "loss": loss,
            "q_values_mean": tf.reduce_mean(self.q_net(batch["obs"])),
            "epsilon": self.policy._epsilon,
            "vars": {key: variable for key, (_, variable) in grads_and_vars.items()},
            "grads": {f"grad_{key}": grad for key, (grad, _) in grads_and_vars.items()},
        }

        return stats

    def save(self):
        """Salva o estado atual do agente (i.e., o valor dos parâmetros da rede online) nesse momento."""
        return self._ckpt_manager.save()

    def restore(self, save_path=None):
        """Carrega o último checkpoint salvo anteriormente no `save_path`."""
        if not save_path:
            save_path = self._ckpt_manager.latest_checkpoint
        return self._ckpt.restore(save_path)

## Protocolo de treinamento, avaliação e teste

Com a classe do `DQN` definida é hora de treinar o agente e avaliá-lo. Faremos isso seguindo um protocolo de treinamento e avaliação definido pelas funções `train` e  `evaluate` abaixo.

<img src="https://github.com/thiagopbueno/curso-verao-rl-ime-2021/blob/master/notebooks/aula3/img/rl-training.png?raw=true"/>

Tente entender como os hiperparâmetros de início de treinamento `learning_starts` e frequência de atualizações `learn_every` e avaliação `evaluation_freq` definem o protocolo.

> **Observação**: note que embora o protocolo abaixo seja bastante genérico, tenha em mente que diferentes trabalhos alteram a maneira como os processos de coleta de dados, aprendizado e avaliação se intercalam.

In [22]:
def train(
    agent,
    env,
    test_env,
    replay,
    logger,
    total_timesteps=20_000,
    learning_starts=1_000,
    learn_every=1,
    evaluation_freq=1_000
):
    timesteps = 0
    episodes = 0
    episode_returns = deque(maxlen=100)

    best_episode_reward_mean = -np.inf

    with trange(total_timesteps, desc="training") as pbar:

        while timesteps < total_timesteps:
            obs = env.reset()
            episode_return = 0.0

            for episode_length in range(1, env.spec.max_episode_steps + 1):

                # collect
                action = agent.step(np.expand_dims(obs, axis=0), training=True)[0]
                next_obs, reward, done, info = env.step(action)

                timesteps += 1
                episode_return += reward

                # add experience to replay buffer
                terminal = done if episode_length < env.spec.max_episode_steps else False
                replay.add(obs, action, reward, terminal, next_obs)

                # training
                if timesteps >= learning_starts and timesteps % learn_every == 0:
                    batch = replay.sample()
                    stats = agent.learn(batch)
                    stats["episode_return_mean"] = np.mean(episode_returns)
                    logger.log(timesteps, stats, label="train") # logging

                # evaluation
                if timesteps % evaluation_freq == 0:
                    stats = evaluate(agent, test_env)
                    logger.log(timesteps, stats, label="evaluation") # logging

                    # checkpointing
                    if stats["episode_return_mean"] > best_episode_reward_mean:
                        agent.save()
                        best_episode_reward_mean = stats["episode_return_mean"]

                if done:
                    break

                obs = next_obs

            episodes += 1
            episode_returns.append(episode_return)

            # logging
            stats = {
                "episodes": episodes,
                "episode_length": episode_length,
                "episode_return": episode_return,
            }
            logger.log(timesteps, stats, label="collect")
            logger.flush()

            pbar.update(episode_length)
            pbar.set_postfix(timesteps=timesteps, episodes=episodes, avg_returns=np.mean(episode_returns) if episode_returns else None)

    # final evaluation
    stats = evaluate(agent, test_env)
    logger.log(timesteps, stats, label="evaluation")
    logger.flush()

Para avaliarmos o agente, utilizaremos o `eval_env` que foi criado como um ambiente paralelizado (contendo `env.num_envs` rodando de forma assíncrona em paralelo).

> **Observação**: Note no código abaixo, como esse tipo de ambiente altera ligeiramente o ciclo de interação agente-ambiente que vimos nas últimas aulas. Para maiores detalhes, consulte a documentação de `gym.vector.make` e o código dos módulos em [https://github.com/openai/gym/tree/master/gym/vector](https://github.com/openai/gym/tree/master/gym/vector).

In [23]:
def evaluate(agent, env):
    total_reward = np.zeros((env.num_envs,))
    episode_length = np.zeros((env.num_envs,))

    obs = env.reset()
    dones = np.array([False] * env.num_envs)

    while not np.all(dones):
        action = agent.step(obs, training=False)
        obs, reward, done, _ = env.step(action)
        total_reward += (1 - dones) * reward
        episode_length += (1 - dones)
        dones = np.logical_or(dones, done)

    return {
        "episode_return_mean": np.mean(total_reward),
        "episode_return_min": np.min(total_reward),
        "episode_return_max": np.max(total_reward),
    }

Execute a célula abaixo para definir um ciclo de interação agente-ambiente para renderizar episódios do agente após o treinamento e então verificar qualitativamente quão boa foi a política que o agente aprendeu.

In [24]:
from collections import namedtuple

TestEpisode = namedtuple("TestEpisode", ["number", "length", "accumulated_reward"])

def test(agent, env, episodes=3, wait=None, render=True, print_results=False):
    results = []

    for episode in range(episodes):
        done = False
        obs = env.reset()

        if render:
            env.render()

        episode_len = 0
        accumulated_reward = 0

        while not done:
            action = agent.step(np.expand_dims(obs, axis=0), training=False)[0]
            obs, reward, done, _ = env.step(action)

            episode_len += 1
            accumulated_reward += reward

            if render:
                env.render()

            if wait:
                time.sleep(wait)

        episode_result = TestEpisode(episode+1, episode_len, accumulated_reward)
        results.append(episode_result)

        if print_results:
          print(f"Episode {episode_result.number}")
          print(f"Length: {episode_result.length}")
          print(f"Accumulated reward: {episode_result.accumulated_reward}")
          print()

    env.close()

    return results

## Treinando DQN no CartPole-v1

Finalmente, temos todo o código necessário para treinarmos o `DQN` no `CartPole-v1`. Antes de iniciarmos o treinamento, execute a célula abaixo para instanciarmos o `tensorboard`, a ferramenta de *logging* e monitoramento do TensorFlow. Consulte a documentação e os tutoriais disponíveis em [https://www.tensorflow.org/tensorboard](https://www.tensorflow.org/tensorboard) para maiores informações.

In [25]:
%load_ext tensorboard
%tensorboard --logdir logs --reload_interval 10

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 21264), started 9 days, 9:01:46 ago. (Use '!kill 21264' to kill it.)

In [26]:
def run(env, eval_env, run_name="default", trials=3,
        total_timesteps=20_000, learning_starts=1_000, evaluation_freq=1_000,
        gamma=0.99, target_update_freq=1000, learning_rate=1e-3,
        policy_start_val=1.0, policy_end_val=0.01, policy_start_step=1_000, policy_end_step=10_000,
        replay_buffer_batch_size=64):
    for _ in range(trials):
        timestamp = datetime.now().strftime("%Y-%m-%d-%H.%M")
        run_id = osp.join(f"dqn-{env.spec.id}-{run_name}".lower(), timestamp)

        logger = logging.TFLogger(run_id, base_dir="logs")

        buffer = replay.ReplayBuffer(env.observation_space, env.action_space, max_size=total_timesteps, batch_size=replay_buffer_batch_size)
        buffer.build()

        agent = DQN(env.observation_space, env.action_space, checkpoint_dir=f"ckpt/{run_id}",
                    gamma=gamma, target_update_freq=target_update_freq, learning_rate=learning_rate,
                    policy_start_val=policy_start_val, policy_end_val=policy_end_val, policy_start_step=policy_start_step, policy_end_step=policy_end_step)

        agent.build()
        agent.compile()

        train(agent, env, eval_env, buffer, logger, total_timesteps=total_timesteps, learning_starts=learning_starts, evaluation_freq=evaluation_freq)

In [27]:
run(env, eval_env, run_name="default", trials=1)

training:   0%|          | 0/20000 [00:00<?, ?it/s]

Agora é só buscar um café esperar o resultado do treinamento. ;)

### Teste do Agente no CartPole-v1

Escolha um dos agentes treinados acima para testar e visualizar seu comportamento com o código abaixo:
> Lembre-se de que, após a conclusão do treinamento, será gerada uma pasta denominada "ckpt" em seu computador, na qual serão armazenadas as políticas que foram aprendidas.

In [28]:
checkpoint_dir = "ckpt/dqn-cartpole-v1-default/2023-11-28-20:02" # altere essa linha para escolher qual checkpoint do agente

agent = DQN(env.observation_space, env.action_space, checkpoint_dir=checkpoint_dir)
agent.build()
agent.restore()

test(agent, test_env, episodes=10, render=False) #render True só funciona localmente

[TestEpisode(number=1, length=9, accumulated_reward=9.0),
 TestEpisode(number=2, length=8, accumulated_reward=8.0),
 TestEpisode(number=3, length=10, accumulated_reward=10.0),
 TestEpisode(number=4, length=9, accumulated_reward=9.0),
 TestEpisode(number=5, length=9, accumulated_reward=9.0),
 TestEpisode(number=6, length=10, accumulated_reward=10.0),
 TestEpisode(number=7, length=24, accumulated_reward=24.0),
 TestEpisode(number=8, length=22, accumulated_reward=22.0),
 TestEpisode(number=9, length=10, accumulated_reward=10.0),
 TestEpisode(number=10, length=27, accumulated_reward=27.0)]

### 💡 Tarefa: Treinando o DQN com outros hiper parâmetros

Esta seção destina-se ao treinamento do agente, seguindo as orientações fornecidas nos slides do EP3 disponíveis no Moodle. A seguir, inclua as células com as especificações das execuções que serão testadas.


In [29]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:
trials = 3

params = {
    'total_timesteps': 20_000,
    'policy_end_step': 10_000,
    'replay_buffer_batch_size': 64
}

run_name=f"default_trials" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

In [30]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:
trials = 3

params = {
    'total_timesteps': 30_000,
    'policy_end_step': 10_000,
    'replay_buffer_batch_size': 64
}

run_name=f"total_timesteps={params['total_timesteps']}" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/30000 [00:00<?, ?it/s]

training:   0%|          | 0/30000 [00:00<?, ?it/s]

training:   0%|          | 0/30000 [00:00<?, ?it/s]

In [31]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:
trials = 3

params = {
    'total_timesteps': 20_000,
    'policy_end_step': 5_000,
    'replay_buffer_batch_size': 64
}

run_name=f"policy_end_step={params['policy_end_step']}" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

In [32]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:
trials = 3

params = {
    'total_timesteps': 20_000,
    'policy_end_step': 10_000,
    'replay_buffer_batch_size': 96
}

run_name=f"replay_buffer_batch_size={params['replay_buffer_batch_size']}" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

training:   0%|          | 0/20000 [00:00<?, ?it/s]

In [33]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:
trials = 3

params = {
    'total_timesteps': 30_000,
    'policy_end_step': 5_000,
    'replay_buffer_batch_size': 96
}

run_name=f"all" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/30000 [00:00<?, ?it/s]

training:   0%|          | 0/30000 [00:00<?, ?it/s]

training:   0%|          | 0/30000 [00:00<?, ?it/s]

In [35]:
# Exemplos de alguns hiper parâmetros que podem ser alterados:

trials = 3

params = {
    "total_timesteps": 10_000,
    "learning_starts": 500,
    "evaluation_freq": 100,
    "target_update_freq": 100,
    "gamma": 0.99,
    "learning_rate": 1e-3,
    "policy_start_val": 1.0,
    "policy_end_val": 0.001,
    "policy_start_step": 1000,
    "policy_end_step": 6000,
    "replay_buffer_batch_size": 128,
}
run_name = f"test" # Altere este nome para melhor organização dos resultados

# Exemplo de execução
run(env, eval_env, run_name, trials, **params)

training:   0%|          | 0/10000 [00:00<?, ?it/s]

training:   0%|          | 0/10000 [00:00<?, ?it/s]

training:   0%|          | 0/10000 [00:00<?, ?it/s]